In [ ]:
%pip install -q -U google-genai

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key="<INSERT YOUR GEMINI API KEY HERE>")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        max_output_tokens=5,
        temperature=0.1
    )
)
print(response.text)

# ------------------------------------- COUNT ------------------------------------

# Brightness

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_brightness_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Brightness/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}."
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Contrast

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_Contrast_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Contrast/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Defocus-blur

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_defoblur_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Defocus-blur/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Elastic

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_elastic_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Elastic/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Fog

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_fog_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Fog/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Frost

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_frost_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Frost/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Gaussian-noise

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_GN_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Gaussian-noise/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Impulse-noise

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_impulse_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Impulse-noise/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# JPEG-compression

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_jpeg_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/JPEG-compression/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Pixelate

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_pixelate_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Pixelate/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Rain

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_rain_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Rain/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Saturation

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_saturation_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Saturation/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Shot-noise

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_shot_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Shot-noise/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Snow

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_snow_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Snow/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Spatter

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_spatter_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Spatter/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Speckle-noise

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_speckle_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Speckle-noise/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Zoom-blur

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_zoom-blur_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Zoom-Blur/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")


# Motion-blur

In [ ]:
import pandas as pd
import os
import json
from PIL import Image
from collections import defaultdict
import google.generativeai as genai

# Configure Gemini API
API_KEY = "<INSERT YOUR GEMINI API KEY HERE>"
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Paths
DATA_PATH = "Noisy-Denoised_QuestionPairs[new].csv"
CHECKPOINT_PATH = "checkpoint_motion-blur_NoisyClean.json"

# Load dataset
data = pd.read_csv(DATA_PATH)

# Load previous progress if exists
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        checkpoint_data = json.load(f)
    
    # Use .get() to prevent KeyError if the key does not exist
    processed_questions = defaultdict(list, checkpoint_data.get("processed_questions", {}))
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0}, checkpoint_data.get("accuracy", {}))
    total_correct = checkpoint_data.get("total_correct", 0)
    total_predictions = checkpoint_data.get("total_predictions", 0)
else:
    processed_questions = defaultdict(list)
    noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    total_correct = 0
    total_predictions = 0


# Iterate over rows where category is "vcr"
for index, row in data.iterrows():
    if row["category"] == "vcr" and row['modified_question_function_name'] == 'misspell_word':
        question_id = row["id"]
        noise_type = row["modified_question_function_name"]

        # Skip if this specific question with this noise type was already processed
        if noise_type in processed_questions[question_id]:
            continue

        image_path = f"Noisy DARE TEST/Motion-blur/{row['path']}"
        
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image not found: {image_path}. Skipping...")
            continue

        # Prepare prompt
        question_type = row["category"]
        question = row["question"]
        options = {key: row[key] for key in ["A", "B", "C", "D"]}
        
        prompt_text = (
            f"The following are multiple-choice questions about {question_type}. "
            "You should directly answer the question by choosing the correct option given the image and the question. "
            "Give only the letter indicating the correct answer e.g., 'A'.\n"
            f"Question: {question}\n"
            "Options:\n"
            f"A. {options['A']}\n"
            f"B. {options['B']}\n"
            f"C. {options['C']}\n"
            f"D. {options['D']}\n"
            "Answer:"
        )
        
        try:
            # Send request to Gemini API
            response = model.generate_content([prompt_text, image])
            full_answer = response.text.strip()
            predicted_answer = full_answer.split()[-1]  # Extract last word

        except Exception as e:
            print(f"API error: {e}. Saving checkpoint and exiting...")
            continue  # Stop execution to prevent further API exhaustion

        # Evaluate accuracy
        if predicted_answer == row["answer"][2]:  # Compare with actual answer
            noise_type_accuracy[noise_type]["correct"] += 1
            total_correct += 1
        noise_type_accuracy[noise_type]["total"] += 1
        total_predictions += 1

        # Mark this (question_id, noise_type) pair as processed
        processed_questions[question_id].append(noise_type)

        # Save checkpoint every 50 iterations
        if total_predictions % 50 == 0:
            checkpoint_data = {
                "processed_questions": dict(processed_questions),
                "accuracy": dict(noise_type_accuracy),
                "total_correct": total_correct,
                "total_predictions": total_predictions,
            }
            with open(CHECKPOINT_PATH, "w") as f:
                json.dump(checkpoint_data, f)
            print(f"Checkpoint saved at {CHECKPOINT_PATH}")

        # Print progress
        print(f"Question ID {row['id']} - Predicted: {predicted_answer} - Actual: {row['answer']} - Noise: {noise_type}")
        print(f"Progress: {total_correct}/{total_predictions}")
        print("Noise Type Accuracy so far:")
        for n_type, counts in noise_type_accuracy.items():
            print(f"{n_type}: {counts['correct']}/{counts['total']}")
        print("----------------------")

# Final checkpoint save
checkpoint_data = {
    "processed_questions": dict(processed_questions),
    "accuracy": dict(noise_type_accuracy),
    "total_correct": total_correct,
    "total_predictions": total_predictions,
}
with open(CHECKPOINT_PATH, "w") as f:
    json.dump(checkpoint_data, f)

print(f"Final checkpoint saved at {CHECKPOINT_PATH}.")
